In [4]:
import pandas as pd
import os
import sqlite3
from sqlite3 import Error
connector = sqlite3.connect('/Users/annaszczepara/Desktop/Code Academy/Project 5/superstore_database.db')

In [5]:
# What is the category generating the maximum sales revenue?
# What about the profit in this category?
# Are they making a loss in any categories?

pd.read_sql("SELECT category, sum(s.sales) as TotalSales FROM sales s JOIN products p ON p.ProductID=s.ProductID group by category order by sales desc", connector)


,Category,TotalSales
0,Technology,519735.4950
1,Furniture,477633.4292
2,Office Supplies,452727.2730


In [6]:
# What are 5 states generating the maximum sales revenue?
pd.read_sql("Select sum(s.sales)as TotalSales,c.state FROM sales s JOIN customers c ON c.CustomerID = s.CustomerID GROUP BY State ORDER BY TotalSales DESC LIMIT 5", connector)

,TotalSales,State
0,287913.3126,California
1,167146.6095,New York
2,135148.4414,Texas
3,93285.6888,Washington
4,82414.7060,Pennsylvania


In [7]:
#...and minimum...?
pd.read_sql("Select sum(s.sales) as TotalSales,c.state FROM sales s JOIN customers c ON c.CustomerID = s.CustomerID GROUP BY State ORDER BY TotalSales ASC LIMIT 5", connector)

,TotalSales,State
0,171.620,Nevada
1,569.608,Kansas
2,1074.414,Maryland
3,1544.504,District of Columbia
4,1557.438,South Dakota


In [8]:
# What are the 3 products in each product segment with the highest sales?
query = """
WITH RankedProducts AS (
    SELECT 
        p.ProductID,
        c.Segment,
        SUM(s.Sales) AS TotalSales,
        RANK() OVER (PARTITION BY c.Segment ORDER BY SUM(s.Sales) DESC) AS rnk
    FROM 
        sales s 
    JOIN 
        customers c ON c.CustomerID = s.CustomerID 
    JOIN 
        Products p ON p.ProductID = s.ProductID 
    GROUP BY 
        p.ProductID, c.Segment
)

SELECT 
    ProductID,
    Segment,
    TotalSales
FROM 
    RankedProducts
WHERE 
    rnk <= 3;
"""

# Execute the query and store the result
top_products_by_segment = pd.read_sql(query, connector)

# Display the result
print(top_products_by_segment)

         ProductID      Segment  TotalSales
0  TEC-CO-10004722     Consumer   18899.946
1  OFF-BI-10000545     Consumer   15752.286
2  FUR-CH-10002024     Consumer   14580.384
3  TEC-CO-10004722    Corporate   17499.950
4  TEC-MA-10004125    Corporate    7999.980
5  TEC-MA-10000822    Corporate    6119.964
6  TEC-CO-10004722  Home Office   11199.968
7  TEC-CO-10001449  Home Office    8279.862
8  TEC-PH-10002885  Home Office    5328.606


In [9]:
# Are they the 3 most profitable products as well?
pd.read_sql("Select * from Sales group by ProductID order by Profit DESC", connector)

,SalesID,CustomerID,ProductID,Sales,Quantity,Discount,Profit
0,5997,SC-20095,OFF-BI-10001120,9449.950,5,0.0,4630.4755
1,6280,TA-21385,TEC-CO-10004722,11199.968,4,0.2,3919.9888
2,165,CM-12385,OFF-BI-10003527,6354.950,5,0.0,3177.4750
3,8395,KD-16270,TEC-MA-10003979,4899.930,7,0.0,2400.9657
4,8938,NM-18445,TEC-MA-10002927,3991.980,2,0.0,1995.9900
...,...,...,...,...,...,...,...
1778,1975,VW-21775,TEC-MA-10000864,1188.000,9,0.7,-950.4000
1779,5269,BM-11140,TEC-MA-10000822,8159.952,8,0.4,-1359.9920
1780,5602,TB-21520,FUR-BO-10004834,3083.430,7,0.5,-1665.0522
1781,9330,NF-18385,TEC-MA-10000418,1799.994,2,0.7,-2639.9912


In [11]:
# What are the 3 products in each product segment with the highest sales?
query = """
WITH RankedProducts AS (
    SELECT 
        p.ProductID,
        c.Segment,
        SUM(s.Sales) AS TotalSales,
        RANK() OVER (PARTITION BY c.Segment ORDER BY SUM(s.Sales) DESC) AS rnk
    FROM 
        sales s 
    JOIN 
        customers c ON c.CustomerID = s.CustomerID 
    JOIN 
        Products p ON p.ProductID = s.ProductID 
    GROUP BY 
        p.ProductID, c.Segment
)

SELECT 
    ProductID,
    Segment,
    TotalSales
FROM 
    RankedProducts
WHERE 
    rnk <= 3;
"""

# Execute the query and store the result
top_products_by_segment = pd.read_sql(query, connector)

# Display the result
print(top_products_by_segment)

         ProductID      Segment  TotalSales
0  TEC-CO-10004722     Consumer   18899.946
1  OFF-BI-10000545     Consumer   15752.286
2  FUR-CH-10002024     Consumer   14580.384
3  TEC-CO-10004722    Corporate   17499.950
4  TEC-MA-10004125    Corporate    7999.980
5  TEC-MA-10000822    Corporate    6119.964
6  TEC-CO-10004722  Home Office   11199.968
7  TEC-CO-10001449  Home Office    8279.862
8  TEC-PH-10002885  Home Office    5328.606


In [16]:
# What are the 3 best-seller products in each product segment? (Quantity-wise)

pd.read_sql("""
WITH RankedProducts AS (
    SELECT 
        p.ProductID,
        c.Segment,
        SUM(s.Quantity) AS TotalQty,
        RANK() OVER (PARTITION BY c.Segment ORDER BY SUM(s.Quantity) DESC) AS rnk
    FROM 
        sales s 
    JOIN 
        customers c ON c.CustomerID = s.CustomerID 
    JOIN 
        Products p ON p.ProductID = s.ProductID 
    GROUP BY 
        p.ProductID, c.Segment
)

SELECT 
    ProductID,
    Segment,
    TotalQty
FROM 
    RankedProducts
WHERE 
    rnk <= 3;
""", connector)

,ProductID,Segment,TotalQty
0,OFF-BI-10000050,Consumer,40
1,OFF-LA-10002762,Consumer,34
2,OFF-BI-10001922,Consumer,33
3,OFF-FA-10002780,Corporate,29
4,OFF-BI-10003291,Corporate,25
5,FUR-CH-10004983,Corporate,24
6,FUR-CH-10002304,Home Office,27
7,OFF-PA-10003441,Home Office,24
8,FUR-FU-10003724,Home Office,19
9,OFF-PA-10001289,Home Office,19


In [14]:
# What are the top 3 worst-selling products in every category? (Quantity-wise)
pd.read_sql("""
WITH RankedProducts AS (
    SELECT 
        p.ProductID,
        c.Segment,
        SUM(s.Quantity) AS TotalQty,
        RANK() OVER (PARTITION BY c.Segment ORDER BY SUM(s.Quantity) ASC) AS rnk
    FROM 
        sales s 
    JOIN 
        customers c ON c.CustomerID = s.CustomerID 
    JOIN 
        Products p ON p.ProductID = s.ProductID 
    GROUP BY 
        p.ProductID, c.Segment
)

SELECT 
    ProductID,
    Segment,
    TotalQty
FROM 
    RankedProducts
WHERE 
    rnk <= 3;
""", connector)

,ProductID,Segment,TotalQty
0,FUR-BO-10002206,Consumer,1
1,FUR-CH-10001482,Consumer,1
2,FUR-CH-10003606,Consumer,1
3,FUR-FU-10000206,Consumer,1
4,FUR-FU-10000246,Consumer,1
...,...,...,...
153,TEC-PH-10003357,Home Office,1
154,TEC-PH-10003601,Home Office,1
155,TEC-PH-10004006,Home Office,1
156,TEC-PH-10004120,Home Office,1


In [11]:
pd.read_sql("Select * from Sales group by ProductID order by Profit DESC", connector)

,SalesID,CustomerID,ProductID,Sales,Quantity,Discount,Profit
0,4833,SC-20095,OFF-BI-10001120,9449.950,5,0.0,4630.4755
1,3153,TA-21385,TEC-CO-10004722,11199.968,4,0.2,3919.9888
2,1348,CM-12385,OFF-BI-10003527,6354.950,5,0.0,3177.4750
3,6372,KD-16270,TEC-MA-10003979,4899.930,7,0.0,2400.9657
4,6473,TB-21400,TEC-MA-10001127,6999.960,4,0.0,2239.9872
...,...,...,...,...,...,...,...
1777,1541,VW-21775,TEC-MA-10000864,1188.000,9,0.7,-950.4000
1778,6499,BM-11140,TEC-MA-10000822,8159.952,8,0.4,-1359.9920
1779,319,TB-21520,FUR-BO-10004834,3083.430,7,0.5,-1665.0522
1780,4185,NF-18385,TEC-MA-10000418,1799.994,2,0.7,-2639.9912


In [4]:
# Plotly-Express

In [5]:
pd.read_sql("Select sum(s.sales) as TotalSales,c.state FROM sales s JOIN customers c ON c.CustomerID = s.CustomerID GROUP BY State ORDER BY TotalSales ASC LIMIT 5", connector)

,TotalSales,State
0,8896.410,Nevada
1,10587.762,Kansas
2,11446.164,Maryland
3,11910.366,Arkansas
4,14035.986,District of Columbia


In [18]:
# How many unique customers per month are there for the year 2016. 
# (There's a catch here: contrary to other 'heavier' RDBMS, SQLite does not support the functions YEAR() or MONTH()
#  to extract the year or the month in a OrderDate. You will have to create two new columns: year and month.)
pd.read_sql("""Select c.CustomerID, s.OrderMonth, s.OrderYear,
            count(distinct c.customerID) as customer_count
            from shipment s
            join products p on p.OrderID = s.OrderID
            join sales sa on sa.ProductID = p.ProductID
            join customers c on c.CustomerID = sa.CustomerID
            where OrderYear = '2016'
            group by OrderMonth
            """, connector)

,CustomerID,OrderMonth,OrderYear,customer_count
0,BT-11680,1,2016,50
1,CB-12415,2,2016,43
2,AH-10690,3,2016,79
3,BF-11005,4,2016,149
4,LO-17170,5,2016,126
5,BF-10975,6,2016,142
6,CM-12190,7,2016,108
7,AC-10615,8,2016,112
8,AA-10315,9,2016,189
9,BH-11710,10,2016,97
